In [1]:
import scipy
import os
import pandas as pd
import numpy as np

from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, find
from tqdm.notebook import tqdm
from collections import defaultdict
from matplotlib import pyplot
from scipy.optimize import minimize

# Подготовка данных

In [2]:
INPUT_FILES_PATH = '../input/netflix-prize-data/'

Удалим пропуски в ID пользователей (исполнение займёт 2-3 минуты)

In [3]:
user_ids = set()
user_ids_parts = []
for file_part in range(1, 5): 
    user_ids_parts.append({line.split(',')[0] for line in open(INPUT_FILES_PATH + f'combined_data_{file_part}.txt') if ':' not in line})
for user_ids_part in user_ids_parts:
    user_ids = user_ids.union(user_ids_part)
user_ids = {old_user_id: new_user_id for new_user_id, old_user_id in enumerate(user_ids)}

In [4]:
is_key_type_string = type(list(user_ids.keys())[0]) == str
is_value_type_int = type(list(user_ids.values())[0]) == int

print(f'В словаре 480189 значений: {len(user_ids)  == 480189}')
print(f'Тип ключей - string: {is_key_type_string}')
print(f'Тип значений - integer: {is_value_type_int}')

В словаре 480189 значений: True
Тип ключей - string: True
Тип значений - integer: True


Считаем данные из исходных файлов (~ 6 минут)

In [5]:
users =   [] # user ids   translated into row indeces (which are 1 less than id)
movies =  [] # movies ids translated into col indeces (which are 1 less than id)
ratings = [] # rating[k] = (rating that users[k] user gave to movies[k] movie)
for file_part in range(1, 5): # the training data is splited into 4 parts
    filename = INPUT_FILES_PATH + f'combined_data_{file_part}.txt'
    with open(filename) as f:
        for line in tqdm(f):
            if ':' in line:
                movie_id = int(line.split(':')[0])
                continue
            (user_id, rating, _) = line.split(',')
            users.append(user_ids[user_id])
            movies.append(int(movie_id) - 1)
            ratings.append(int(rating))

Запишем их в единый csv файл

In [6]:
with open('data_csv.csv', mode='w+') as f:
    f.write('video_id,user_id,rating\n')
    for i in range(len(users)):
        f.write(f'{users[i]},{movies[i]},{ratings[i]}\n')

Почистим память от исходных данных, которые мы уже записали в более удобном виде в csv файл

In [7]:
del users
del movies
del ratings

Загрузим данные в pandas dataframe

In [8]:
user_movie_rating = pd.read_csv('data_csv.csv')
user_movie_rating.head()

,video_id,user_id,rating
0,244197,0,3
1,219652,0,5
2,133800,0,4
3,101743,0,4
4,75717,0,3


Возьмём только первую 1000 фильмов по количеству полученных оценок и первую 1000 пользователей по количеству поставленных оценок

In [9]:
most_ratings_movies = user_movie_rating['video_id'].value_counts()[:1000].index
most_ratings_users =  user_movie_rating['user_id'] .value_counts()[:1000].index

user_movie_rating_top_videos = user_movie_rating[user_movie_rating['video_id'].isin(most_ratings_movies)]
user_movie_rating_top = user_movie_rating_top_videos[user_movie_rating_top_videos['user_id'].isin(most_ratings_users)]
del user_movie_rating_top_videos
user_movie_rating_top.head()

,video_id,user_id,rating
52567,470216,27,1
52579,335513,27,4
52643,133837,27,4
52727,282650,27,4
52755,440689,27,4


Создадим новые ID (чтобы не было столбцов и строк с индексом выше 1000 и наша sparse матрица не была слишком большой)

In [10]:
new_movie_ids = dict()
current_movie_id = 0
for old_movie_id in user_movie_rating_top['video_id'].unique():
    new_movie_ids[old_movie_id] = current_movie_id
    current_movie_id+=1
    
new_user_ids = dict()
current_user_id = 0
for old_user_id in user_movie_rating_top['user_id'].unique():
    new_user_ids[old_user_id] = current_user_id
    current_user_id+=1

И поменяем их ID в нашей таблице

In [11]:
user_movie_rating_top['user_id'] = user_movie_rating_top['user_id'].apply(lambda x: new_user_ids[x])
user_movie_rating_top['video_id'] = user_movie_rating_top['video_id'].apply(lambda x: new_movie_ids[x])
user_movie_rating_top.head()

,video_id,user_id,rating
52567,0,0,1
52579,1,0,4
52643,2,0,4
52727,3,0,4
52755,4,0,4


Посмотрим, сколько значений матрицы не будут пустыми

In [12]:
print(f'{100*len(user_movie_rating_top)/(1000*1000):.0f}%')

75%


Создадим саму матрицу. Каждая строка представляет из себя видео, а каждый столбец -- юзера. В способе с разложением матрицы это не имеет значения, но для user_based и item_based коллаборативной фильтрации, а именно для применения KNN, мы должны понимать, что будет выступать в роли признаков, а что в роли объектов.

In [13]:
ratings_matrix = csr_matrix((user_movie_rating_top['rating'], (user_movie_rating_top['video_id'], user_movie_rating_top['user_id'])))
ratings_matrix.shape

(1000, 1000)

Выделим тестовый сет из этой матрицы
Представленный далее алгоритм удаляет максимум один элемент из каждой строки и столбца. Таким образом, максимум будет удалено min(N, M) значений

In [14]:
def extract_test_dataset():
    deleted_user_ids, deleted_movie_ids, deleted_ratings = [], [], []
    
    count_movies = user_movie_rating_top['video_id'].value_counts()
    count_users = user_movie_rating_top['user_id'].value_counts()
    
    movie_ids, user_ids, ratings = np.array(find(ratings_matrix))
    for movie_id, user_id, rating in tqdm(np.array(find(ratings_matrix)).T):
        if count_movies.get(movie_id) > 1 and count_users.get(user_id) > 1:
            if movie_id not in deleted_movie_ids and user_id not in deleted_user_ids:
                deleted_user_ids.append(user_id)
                deleted_movie_ids.append(movie_id)
                deleted_ratings.append(rating)
    
    for i in range(len(deleted_user_ids)):
        ratings_matrix[deleted_movie_ids[i], deleted_user_ids[i]] = 0
        
    return deleted_user_ids, deleted_movie_ids, deleted_ratings

test_user_ids, test_movie_ids, test_ratings = extract_test_dataset()

Проверка того, что значения удалились

In [15]:
for test_rating_number in range(len(test_user_ids)):
    assert ratings_matrix[int(test_movie_ids[test_rating_number]), int(test_user_ids[test_rating_number])] == 0 
    assert test_ratings[test_rating_number] != 0

# Обучение модели. Вариант 1: разложение матрицы при помощи готового SVD

### Исследования проводились на подвыборке

Примерно 6 минут

In [17]:
from scipy.sparse.linalg import svds

# csr_matrix.mean(ratings_matrix, axis=???)
def fill_user_rating_table(ratings_matrix, k):
    ratings_matrix = ratings_matrix.astype('f')
    U, sigma, Vt = svds(ratings_matrix, k=k)
    sigma = np.diag(sigma)
    return np.dot(np.dot(U, sigma), Vt)

# Подсчёт средней ошибки
def compute_err(trained_user_rating_matrix):
    mean_err = 0
    for test_rating_number in range(len(test_user_ids)):
        mean_err += (abs(trained_user_rating_matrix[int(test_movie_ids[test_rating_number]), int(test_user_ids[test_rating_number])] - test_ratings[test_rating_number]))
    return mean_err / len(test_user_ids)

def compute_err_real_estimation(trained_user_rating_matrix):
    mean_err = 0
    for test_rating_number in range(len(test_user_ids)):
        rating_estimation = trained_user_rating_matrix[int(test_movie_ids[test_rating_number]), int(test_user_ids[test_rating_number])]
        if rating_estimation < 1:            
            real_rating_estimation = 1
        elif rating_estimation > 5:
            real_rating_estimation = 5
        else:
            real_rating_estimation = int(round(rating_estimation, 0))
        mean_err += abs(real_rating_estimation - test_ratings[test_rating_number])
    return mean_err / len(test_user_ids)


# trained_user_rating_matrix = fill_user_rating_table(ratings_matrix, 14)
print(compute_err(trained_user_rating_matrix))
print(compute_err_real_estimation(trained_user_rating_matrix))

0.9496705532055311
0.9198396793587175


Код выше был применён на всех k от 1 до 200 включительно. График имел локальный минимум при k = 14 с ошибкой ~0.95 в среднем. При k < 14 функция ошибки монотонно убывает, а после него монотонно возрастает.

Если оптимизировать функцию таким образом, чтобы она считала разницу не между значением матрицы предсказаний и реальным значением рейтинга, а строила по предсказанию оценку, то ошибка немного упадёт до ~0.92.

Для этого исследования была взята подвыборка, которая имела больше всего значений (иначе говоря, подматрица 1000 * 1000 с самым маленьким количеством пустых значений) и обучение дало приемлимый результат (средняя ошибка всего 0.94 балла). При этом на полном наборе данных (в другом jupyter notebook) ошибка составила около 2.5.
В полном датасете 100 миллионов значений при 8 миллиардов сочетаний user-movie. Таким образом, в полном датасете полнота составляет 1% (в то время как в этом сабсете 75%). Можно сделать вывод, что полнота имеет влияние на обучение, полный датасет предсказывает значения с огрмной средней ошибкой (2.5 при оценках от 1 до 5).


# Обучение модели. Вариант 2: KNN item-based

Первый вариант:
* Объекты -- видео
* Признаки -- пользователи
В этом случае можно будет подавать на вход модели видео и выдавать похожие видео.

По следам [статьи](https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea) были взяты аналогичные параметры (см. параметры NearestNeighbors)

In [70]:
from sklearn.neighbors import NearestNeighbors
knn_item = NearestNeighbors(n_neighbors=20, metric='cosine', algorithm='brute', n_jobs=-1)
item_based_knn = knn_item.fit(ratings_matrix)

Теперь можно предсказывать, какую оценку поставит пользователь конкретному видео. Возьмём одно значение из тестовой выборки, чтобы понять процесс

In [27]:
u_id, m_id, rat = test_user_ids[0], test_movie_ids[0], test_ratings[0]
# Индексы похожих фильмов
_, idxs = item_based_knn.kneighbors(ratings_matrix[m_id, :])
# Возвращаем самый похожий
most_alike_movie = ratings_matrix[idxs[:, 1], :]
most_alike_movie[:, u_id][0, 0]

0

Сразу же можно отметить проблему этого метода: пользователь попросту мог не посмотреть похожие видео и оценка будет 0. Однако полностью исключить использоавние этого метода по этой причине нельзя. Попытаемся исправить ситуацию просмотром не одного, а нескольких близжайших фильмов

In [28]:
for idx in idxs[:, 1:]:
    most_alike_movie = ratings_matrix[idx, :]
    if most_alike_movie[:, u_id][0, 0] != 0:
        print(most_alike_movie[:, u_id][0, 0])
        break

Однако такого не нашлось. Посмотрим на картину в целом и посчитаем ошибку несколькими методами.

1. Считаем только разницу между предсказаным и реальным значением, если предсказаное значение существует (то есть не равно 0).
1. Считаем разницу между предсказанным и реальным значением даже если предсказали 0

Помимо этого посчитаем, сколько значений не было предсказано совсем.

In [71]:
def get_prediction(item_based_knn, u_id, m_id):
    _, idxs = item_based_knn.kneighbors(ratings_matrix[m_id, :])
    for idx in idxs[0, 1:]:
        most_alike_movie = ratings_matrix[idx, :]
        if most_alike_movie[:, u_id][0, 0] != 0:
            return most_alike_movie[:, u_id][0, 0]
    return 0



def compute_err(test_us, test_ms, real_rating):
    total_err = []
    predicted_err = []
    count_not_predicted = 0
    for i in tqdm(range(len(test_us))):
        prediction = get_prediction(item_based_knn, test_us[i], test_ms[i])
        if prediction == 0:
            count_not_predicted+=1
        else:
            predicted_err.append(abs(prediction - real_rating[i]))
        total_err.append(abs(prediction - real_rating[i]))
        
    return sum(total_err)/len(total_err), sum(predicted_err)/len(predicted_err), count_not_predicted

compute_err(test_user_ids, test_movie_ids, test_ratings)

(0.9649298597194389, 0.9649298597194389, 0)

Ошибка в этом подходе составляет 0.96. При этом для всех значений нашлось предсказание. Причина этому -- очень хороший датасет, который мы выделили из общей выборки.

# Обучение модели. Вариант 3: KNN user-based

Всё, что поменяется в этом случае - это матрица, а именно мы транспонируем её, чтобы объектами теперь были пользователи, а не фильмы

In [72]:
knn_user = NearestNeighbors(n_neighbors=20, metric='cosine', algorithm='brute', n_jobs=-1)
user_based_knn = knn_user.fit(ratings_matrix.T)

Снова сделаем методы для предсказания и подсчёта ошибки подобным образом

In [73]:
def get_prediction_user(user_based_knn, u_id, m_id):
    _, idxs = user_based_knn.kneighbors(ratings_matrix.T[u_id, :])
    for idx in idxs[0, 1:]:
        most_alike_user = ratings_matrix[:, idx]
        if most_alike_user[m_id, :][0, 0] != 0:
            return most_alike_user[m_id, :][0, 0]
    return 0

def compute_err_user(test_us, test_ms, real_rating):
    total_err = []
    predicted_err = []
    count_not_predicted = 0
    for i in tqdm(range(len(test_us))):
        prediction = get_prediction_user(user_based_knn, test_us[i], test_ms[i])
        if prediction == 0:
            count_not_predicted+=1
        else:
            predicted_err.append(abs(prediction - real_rating[i]))
        total_err.append(abs(prediction - real_rating[i]))
        
    return sum(total_err)/len(total_err), sum(predicted_err)/len(predicted_err), count_not_predicted

compute_err_user(test_user_ids, test_movie_ids, test_ratings)

(0.7605210420841684, 0.7605210420841684, 0)

Ошибка в этом подходе составляет 0.76. Опять же, для всех значений нашлось предсказание.

# Вариант 4: KNN user-based & item-based

Попробуем совместить подходы. Для этого будем считать оценку двумя методами, а потом брать их взвешенное среднее. В качестве весов возьмём обратное значения пропорциональные обрантым ошибкам на тестовых выборок для обоих подходов:

\\(w_1, w_2\\) - веса оценки первого и второго подхода

\\(x, y\\)     - ошибка первого и второго метода

\\(w_1/w_2 = y/x, w_1+w_2=2\\) - соотношение, которого пытаемся добиться

\\(w_1 = 2 / (1 + (x/y)), w_2 = x/y * w_1)\\) -- решение системы

In [79]:
x = 0.9649298597194389
y = 0.7605210420841684

w1 = 2/(1+(x/y))
w2 = (x/y)*w1

def get_prediction_user_movie(item_based_knn, user_based_knn, u_id, m_id):    
    r1 = get_prediction(item_based_knn, u_id, m_id)    
    r2 = get_prediction_user(user_based_knn, u_id, m_id)
    
    return (r1*w1 + r2*w2)/2

def compute_err_user_movie(test_us, test_ms, real_rating):
    total_err = []
    predicted_err = []
    count_not_predicted = 0
    for i in tqdm(range(len(test_us))):
        prediction = get_prediction_user_movie(item_based_knn, user_based_knn, test_us[i], test_ms[i])
        prediction = int(round(prediction, 0))
        if prediction == 0:
            count_not_predicted+=1
        else:
            predicted_err.append(abs(prediction - real_rating[i]))
        total_err.append(abs(prediction - real_rating[i]))
    return sum(total_err)/len(total_err), sum(predicted_err)/len(predicted_err), count_not_predicted
        
compute_err_user_movie(test_user_ids, test_movie_ids, test_ratings)

(0.7244488977955912, 0.7244488977955912, 0)

In [80]:
w1, w2

(0.881533101045296, 1.1184668989547037)

Совмещённый подход выдал наилучший результат: 0.72. Веса для user_based и item_based были 1.1184668989547037, 0.881533101045296 соответственно. Для этого подхода нужно было прогнать алгоритм на тестовой выборке для каждого подхода отдельно, чтобы получить такие веса.

# Вариант 5: Mixed

В качестве финального варианта применим все методы вместе. Веса опять вычислим по ошибкам на тестовом датасете.

In [82]:
x = 0.9649298597194389
y = 0.7605210420841684
z = 0.9496705532055311

w3 = 3*z/(x+y+z)
w2 = (y/z)*w3
w1 = (x/z)*w3

def get_prediction_user_movie_svd(item_based_knn, user_based_knn, trained_user_rating_matrix, u_id, m_id):    
    r1 = get_prediction(item_based_knn, u_id, m_id)    
    r2 = get_prediction_user(user_based_knn, u_id, m_id)
    r3 = trained_user_rating_matrix[m_id, u_id]    
    return (r1*w1 + r2*w2 + r3*w3)/3

def compute_err_user_movie_svd(test_us, test_ms, real_rating):
    total_err = []
    predicted_err = []
    count_not_predicted = 0
    for i in tqdm(range(len(test_us))):
        prediction = get_prediction_user_movie_svd(item_based_knn, user_based_knn, trained_user_rating_matrix, test_us[i], test_ms[i])
        prediction = int(round(prediction, 0))
        if prediction == 0:
            count_not_predicted+=1
        else:
            predicted_err.append(abs(prediction - real_rating[i]))
        total_err.append(abs(prediction - real_rating[i]))
    return sum(total_err)/len(total_err), sum(predicted_err)/len(predicted_err), count_not_predicted
        
compute_err_user_movie_svd(test_user_ids, test_movie_ids, test_ratings)

(0.6933867735470942, 0.6933867735470942, 0)

In [83]:
w1, w2, w3

(1.082115196578403, 0.8528820708234764, 1.0650027325981208)

В финальном варианте, где использовались все три метода вместе, получилась средняя ошибка = 0.69. Веса для методов были взяты следующие: 1.082115196578403, 0.8528820708234764, 1.0650027325981208

# Выводы

Безусловно, достичь лучшего результата можно при помощи комбинирования всех трёх методов вместе. Наименьшую ошибку на маленькой выборке показал метод user_based. При этом самая маленькая ошибка достигается при применении всех методов вместе с весами обратно пропорциональными ошибкам на тестовой выборке. Замеченные плюсы и минусы каждого метода:

**SVD**

Плюсы:
1. Быстрый поиск предсказания для user/video
2. Гарантия наличия предсказания для любой пары user/video

Минусы:
1. Обучение дольше, чем в случае с KNN
2. Самая большая ошибка (на маленькой выборке); на большой выборке ошибка достигла 2.5 на оптимальном значении k. Такие предсказания будут довольно неточными. Когда мы говорим о рейтинге от 1 до 5, погрешность 2.5 в среднем перевеводит фильм из категории "не рекомендовать" в категорию "рекомендовать" и наоборот, что не является удовлетворительным результатом для использоавния системы на практике
3. Чтобы хранить матрицу предсказаний на большой выборке понадобится достаточно много памяти. Для предложенного датасета netflix prize понадобится 32ГБ (так как значения записаны в формате float32, а матрица имеет размеры 17770 (фильмов) и 480189 (пользователей) получаем выражение 17770\*480189\*32/(8\*(1024\*\*3))~32ГБ). Это ограничение в лабораторных условиях, имеющихся в рамках проведённой работы, удалось преодалеть тем, что финальная матрица предсказаний не хранилась, а хранились только три матрицы, полученные в результате SVD, которые при введённом user_id и movie_id перемножались с соответствующей строкой первой матрицы и столбцом третьей матрицы. Другой способ -- хранить значения не во float32, а в byte (8bit), что снизит объёмы памяти в 4 раза (до 8ГБ), однако в рамках этой работы использовался python и в нём достаточно трудно оперировать типами данных (и это, скорее, выходит за рамки поставленных задач).

**KNN**

Абсолютно симметричные плюсы и минусы.

Плюсы:
1. Очень быстрое обучение
2. Ошибка ниже, чем в SVD
3. Обученная модель не требует много памяти для хранения

Минусы:
1. Поиск предсказания на полной выборке настолько долгий, что для всей тестовой выборки (20 миллионов объектов) он занял бы 15,000 часов (625 суток, почти два года). В таком виде этот метод совсем неприменим в продакшене, его нужно оптимизировать. Предложение по оптимизации: вместо KNN использовать какой-то нибудь алгоритм кластеризации для фильмов и пользователей отдельно. В этом случае уже будет иметь место бинарная таргет-метка "рекомендовать или нет".
2. Не можем гарантировать, что оценка будет рекомендована.

В таком виде на данных Netflix применим только первый подход, однако это не заставляет сбрасывать со счетов способ с KNN -- если выборка не слишком большая, то он сильно увеличит точность предсказаний.